In [184]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

batch_size = 1

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/home/liam/Documents/notebook/files', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size, shuffle=True)

class Spike1d:
    def __init__(self, n, pot = 1):
        self.potentials = torch.randn(n, dtype=torch.float)
        self.potentials -= self.potentials
        self.pot = pot
    
    def func(self, a):
        return 2*a
    
    #integral of func
    def func_int(self, a):
        return a^2
    
    def __call__(self, x):
        
        #x has n rows, (n of input layer) columns
        
        if x.shape != self.potentials.shape:
            raise Exception("Size mismatch {} x {}".format(x.shape,self.potentials.shape))
        
        x = x.unsqueeze(-1)
        
        #calculate potential change
        delta_p = torch.sum(func_int(x),-1)
        self.potentials += delta_p
        
        #generate spikes
        spikes = (self.potentials > self.pot).float()
        self.potentials -= (spikes * self.pot)
        print(self.potentials)
        
        return spikes
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.spike1 = Spike1d(2)
        self.fc1 = nn.Linear(2,5,bias = False)
        self.spike2 = Spike1d(5)
        self.fc2 = nn.Linear(5,2,bias = False)
        self.spike3 = Spike1d(2)
    def forward(self, x):
        x = self.spike1(x)
        print(x)
        print()
        x = self.fc1(x)
        x = self.spike2(x)
        print(x)
        print()
        
        x = self.fc2(x)#.unsqueeze(-1)
        x = self.spike3(x)
        print(x)
        return
#network = Net()


#out = network(torch.tensor([0.3]*2, dtype=torch.float))#torch.randn(1, dtype=torch.float))


In [185]:
model = Net()

for i in range(1500):

    out = model(torch.tensor([0.5**(1/2)]*2, dtype=torch.float))#torch.randn(1, dtype=torch.float))
    print("#########")
    

tensor([0.5000, 0.5000])
tensor([0., 0.])

torch.Size([5])
tensor([0., 0., 0., 0., 0.], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 0., 0.])

tensor([0., 0.], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([1.0000, 1.0000])
tensor([0., 0.])

torch.Size([5])
tensor([0., 0., 0., 0., 0.], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 0., 0.])

tensor([0., 0.], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([0.5000, 0.5000])
tensor([1., 1.])

torch.Size([5])
tensor([0.4327, 0.0189, 0.1648, 0.7523, 0.1347], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 0., 0.])

tensor([0., 0.], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([1.0000, 1.0000])
tensor([0., 0.])

torch.Size([5])
tensor([0.4327, 0.0189, 0.1648, 0.7523, 0.1347], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 0., 0.])

tensor([0., 0.], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([0.5000, 0.5000])
tensor([1., 1.])

torch.Size([5])
tensor([0.8654, 0.0378, 0.3297, 0.5045, 0.2695], grad

tensor([0.9642, 0.2267], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([0.5000, 0.5000])
tensor([1., 1.])

torch.Size([5])
tensor([0.2013, 0.9253, 0.0768, 0.8608, 0.6016], grad_fn=<ThSubBackward>)
tensor([1., 0., 1., 0., 0.])

tensor([0.9757, 0.4247], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([1.0000, 1.0000])
tensor([0., 0.])

torch.Size([5])
tensor([0.2013, 0.9253, 0.0768, 0.8608, 0.6016], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 0., 0.])

tensor([0.9757, 0.4247], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([0.5000, 0.5000])
tensor([1., 1.])

torch.Size([5])
tensor([0.6340, 0.9441, 0.2416, 0.6131, 0.7363], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 1., 0.])

tensor([0.9766, 0.5173], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([1.0000, 1.0000])
tensor([0., 0.])

torch.Size([5])
tensor([0.6340, 0.9441, 0.2416, 0.6131, 0.7363], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 0., 0.])

tensor([0.9766, 0.5173], grad_fn=<ThSubB

tensor([0., 0., 0., 0., 0.])

tensor([0.0933, 0.2977], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([1.0000, 1.0000])
tensor([0., 0.])

torch.Size([5])
tensor([0.8549, 0.3029, 0.3734, 0.9061, 0.2961], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 0., 0.])

tensor([0.0933, 0.2977], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([0.5000, 0.5000])
tensor([1., 1.])

torch.Size([5])
tensor([0.2876, 0.3218, 0.5382, 0.6583, 0.4308], grad_fn=<ThSubBackward>)
tensor([1., 0., 0., 1., 0.])

tensor([0.1755, 0.2983], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([1.0000, 1.0000])
tensor([0., 0.])

torch.Size([5])
tensor([0.2876, 0.3218, 0.5382, 0.6583, 0.4308], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 0., 0.])

tensor([0.1755, 0.2983], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([0.5000, 0.5000])
tensor([1., 1.])

torch.Size([5])
tensor([0.7203, 0.3407, 0.7031, 0.4106, 0.5656], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 1., 0.])

tensor([0.

tensor([0.6279, 0.6292], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([1.0000, 1.0000])
tensor([0., 0.])

torch.Size([5])
tensor([0.7007, 0.9072, 0.6480, 0.9785, 0.6074], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 0., 0.])

tensor([0.6279, 0.6292], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([0.5000, 0.5000])
tensor([1., 1.])

torch.Size([5])
tensor([0.1334, 0.9261, 0.8129, 0.7307, 0.7421], grad_fn=<ThSubBackward>)
tensor([1., 0., 0., 1., 0.])

tensor([0.7102, 0.6297], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([1.0000, 1.0000])
tensor([0., 0.])

torch.Size([5])
tensor([0.1334, 0.9261, 0.8129, 0.7307, 0.7421], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 0., 0.])

tensor([0.7102, 0.6297], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([0.5000, 0.5000])
tensor([1., 1.])

torch.Size([5])
tensor([0.5661, 0.9449, 0.9777, 0.4830, 0.8768], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 1., 0.])

tensor([0.7111, 0.7223], grad_fn=<ThSubB

tensor([0.4889, 0.2471, 0.6150, 0.5192, 0.0324], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 0., 0.])

tensor([0.6927, 0.8758], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([0.5000, 0.5000])
tensor([1., 1.])

torch.Size([5])
tensor([0.9216, 0.2659, 0.7798, 0.2714, 0.1672], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 1., 0.])

tensor([0.6936, 0.9684], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([1.0000, 1.0000])
tensor([0., 0.])

torch.Size([5])
tensor([0.9216, 0.2659, 0.7798, 0.2714, 0.1672], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 0., 0.])

tensor([0.6936, 0.9684], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([0.5000, 0.5000])
tensor([1., 1.])

torch.Size([5])
tensor([0.3543, 0.2848, 0.9447, 0.0237, 0.3019], grad_fn=<ThSubBackward>)
tensor([1., 0., 0., 1., 0.])

tensor([0.7759, 0.9690], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([1.0000, 1.0000])
tensor([0., 0.])

torch.Size([5])
tensor([0.3543, 0.2848, 0.9447, 0.0237, 

...

tensor([1.0000, 1.0000])
tensor([0., 0.])

torch.Size([5])
tensor([0.1951, 0.7444, 0.7894, 0.0371, 0.6593], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 0., 0.])

tensor([0.8742, 0.8078], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([0.5000, 0.5000])
tensor([1., 1.])

torch.Size([5])
tensor([0.6277, 0.7632, 0.9543, 0.7894, 0.7940], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 0., 0.])

tensor([0.8742, 0.8078], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([1.0000, 1.0000])
tensor([0., 0.])

torch.Size([5])
tensor([0.6277, 0.7632, 0.9543, 0.7894, 0.7940], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 0., 0.])

tensor([0.8742, 0.8078], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([0.5000, 0.5000])
tensor([1., 1.])

torch.Size([5])
tensor([0.0604, 0.7821, 0.1191, 0.5416, 0.9288], grad_fn=<ThSubBackward>)
tensor([1., 0., 1., 1., 0.])

tensor([0.8801, 0.8276], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([1.0000, 1.0000])
tensor([0., 0.]

tensor([0.5000, 0.5000])
tensor([1., 1.])

torch.Size([5])
tensor([0.0984, 0.6130, 0.3717, 0.6412, 0.8567], grad_fn=<ThSubBackward>)
tensor([1., 0., 0., 1., 0.])

tensor([0.7653, 0.8475], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([1.0000, 1.0000])
tensor([0., 0.])

torch.Size([5])
tensor([0.0984, 0.6130, 0.3717, 0.6412, 0.8567], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 0., 0.])

tensor([0.7653, 0.8475], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([0.5000, 0.5000])
tensor([1., 1.])

torch.Size([5])
tensor([0.5310, 0.6319, 0.5365, 0.3934, 0.9915], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 1., 0.])

tensor([0.7662, 0.9401], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([1.0000, 1.0000])
tensor([0., 0.])

torch.Size([5])
tensor([0.5310, 0.6319, 0.5365, 0.3934, 0.9915], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 0., 0.])

tensor([0.7662, 0.9401], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([0.5000, 0.5000])
tensor([1., 1.]

tensor([0., 0., 0., 0., 0.])

tensor([0.3474, 0.1570], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([0.5000, 0.5000])
tensor([1., 1.])

torch.Size([5])
tensor([0.4056, 0.3683, 0.9650, 0.7316, 0.2458], grad_fn=<ThSubBackward>)
tensor([1., 0., 0., 1., 0.])

tensor([0.4297, 0.1576], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([1.0000, 1.0000])
tensor([0., 0.])

torch.Size([5])
tensor([0.4056, 0.3683, 0.9650, 0.7316, 0.2458], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 0., 0.])

tensor([0.4297, 0.1576], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([0.5000, 0.5000])
tensor([1., 1.])

torch.Size([5])
tensor([0.8382, 0.3872, 0.1298, 0.4839, 0.3805], grad_fn=<ThSubBackward>)
tensor([0., 0., 1., 1., 0.])

tensor([0.5404, 0.1925], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([1.0000, 1.0000])
tensor([0., 0.])

torch.Size([5])
tensor([0.8382, 0.3872, 0.1298, 0.4839, 0.3805], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 0., 0.])

tensor([0.

tensor([0., 0.])

torch.Size([5])
tensor([0.4147, 0.0669, 0.0638, 0.5653, 0.2307], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 0., 0.])

tensor([0.0899, 0.8033], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([0.5000, 0.5000])
tensor([1., 1.])

torch.Size([5])
tensor([0.8474, 0.0858, 0.2286, 0.3176, 0.3654], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 1., 0.])

tensor([0.0909, 0.8959], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([1.0000, 1.0000])
tensor([0., 0.])

torch.Size([5])
tensor([0.8474, 0.0858, 0.2286, 0.3176, 0.3654], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 0., 0.])

tensor([0.0909, 0.8959], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([0.5000, 0.5000])
tensor([1., 1.])

torch.Size([5])
tensor([0.2801, 0.1047, 0.3934, 0.0699, 0.5001], grad_fn=<ThSubBackward>)
tensor([1., 0., 0., 1., 0.])

tensor([0.1731, 0.8965], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([1.0000, 1.0000])
tensor([0., 0.])

torch.Size([5])
tensor

tensor([0.8465, 0.2686], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([1.0000, 1.0000])
tensor([0., 0.])

torch.Size([5])
tensor([0.8853, 0.9167, 0.4812, 0.4171, 0.2933], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 0., 0.])

tensor([0.8465, 0.2686], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([0.5000, 0.5000])
tensor([1., 1.])

torch.Size([5])
tensor([0.3180, 0.9356, 0.6461, 0.1694, 0.4281], grad_fn=<ThSubBackward>)
tensor([1., 0., 0., 1., 0.])

tensor([0.9287, 0.2692], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([1.0000, 1.0000])
tensor([0., 0.])

torch.Size([5])
tensor([0.3180, 0.9356, 0.6461, 0.1694, 0.4281], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 0., 0.])

tensor([0.9287, 0.2692], grad_fn=<ThSubBackward>)
tensor([0., 0.])
#########
tensor([0.5000, 0.5000])
tensor([1., 1.])

torch.Size([5])
tensor([0.7507, 0.9544, 0.8109, 0.9217, 0.5628], grad_fn=<ThSubBackward>)
tensor([0., 0., 0., 0., 0.])

tensor([0.9287, 0.2692], grad_fn=<ThSubB